<a href="https://colab.research.google.com/github/minghui-i/circular_motion_classifier/blob/main/circular_motion_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import re
from time import time

import numpy as np
import pandas as pd

import torch

import torch.nn as nn
from torch.nn import functional as F

from torch.utils.data import Dataset, DataLoader

In [ ]:
class FramesDataset(Dataset):
  def __init__(self, data_pd):
    self.data_pd = data_pd

  def __len__(self):
    return len(self.data_pd)

  def __getitem__(self, index):
    return self.data_pd.iloc[index]['Tensor'], self.data_pd.iloc[index]['Label']

In [ ]:
os.chdir('/content/drive/MyDrive/Object Detection/TrainTensors')
data_df = torch.load('AugementTrain.pt')

In [ ]:
for x in range(len(data_df['Label'])):
  if data_df['Label'].iloc[x] == 'False':
    data_df['Label'].iloc[x] = 0
  else:
    data_df['Label'].iloc[x] = 1

In [ ]:
os.chdir('/content/drive/MyDrive/Object Detection/TestTensors')
data_df_test = torch.load("AugementTester.pt")

In [ ]:
for x in range(len(data_df_test['Label'])):
  if data_df_test['Label'].iloc[x] == '0' or 0:
    data_df_test['Label'].iloc[x] = 0
  elif data_df_test['Label'].iloc[x] == '1' or 1:
    data_df_test['Label'].iloc[x] = 1

In [ ]:
train_dataset = FramesDataset(data_df)
train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=4, pin_memory=True)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [ ]:
test_dataset = FramesDataset(data_df_test)
test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=True, num_workers=4, pin_memory=True)

In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 4, 5, 2)
        self.conv2 = nn.Conv2d(4, 16, 5, 2)
        self.fc1 = nn.Linear(576, 550)
        self.fc2 = nn.Linear(550, 540)
        self.fc3 = nn.Linear(540, 470)
        self.fc4 = nn.Linear(470, 300)
        self.fc5 = nn.Linear(300, 220)
        self.fc6 = nn.Linear(220, 130)
        self.fc7 = nn.Linear(130, 70)
        self.fc8 = nn.Linear(70, 40)
        self.fc9 = nn.Linear(40, 1)
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 3, 3)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 3, 3)
        x = x.flatten(start_dim=1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        x = self.fc3(x)
        x = self.fc4(x)
        x = self.fc5(x)
        x = self.fc6(x)
        x = self.fc7(x)
        x = self.fc8(x)
        x = self.fc9(x)
        return x

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
model = CNN().to(device)

cuda


In [ ]:
# Hyperparameters
epochs = 2
learning_rate = .0001
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

param_count = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'The model has {param_count:,} trainable parameters')

The model has 1,119,181 trainable parameters


In [ ]:
model = torch.compile(model, mode="reduce-overhead")
torch.set_float32_matmul_precision("high")
torch.backends.cudnn.benchmark = True

In [ ]:
for batch_idx, (data, targets) in enumerate(train_dataloader):
    outputs = model(data.to(device)).squeeze(1)
    loss = criterion(outputs, targets.to(device).float())
    break

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [ ]:
times = []
losses = []

for epoch in range(epochs):
    for batch_idx, (data, targets) in enumerate(train_dataloader):

        start = time()
        outputs = model(data.to(device)).squeeze(1)
        loss = criterion(outputs, targets.to(device).float())

        optimizer.zero_grad()

        loss.backward()
        optimizer.step()


        losses.append(loss.item())
        times.append(time() - start)

        msg =  f'Epoch: {epoch+1:02}/{epochs:02} | '
        msg += f'Batch: {batch_idx+1:05}/{len(train_dataloader):05} | '
        msg += f'Loss: {losses[-1]:0.4} | '
        msg += f'Avg Loss: {sum(losses)/len(losses):0.4} | '
        msg += f'Prediction: {outputs[0]} | '
        msg += f'Label: {targets[0]} | '
        msg += f'Time: {times[-1]:0.4} | '
        avg_time = sum(times)/len(times)
        msg += f'Avg Time: {avg_time:0.4} | '
        msg += f'Time Left: {avg_time * (len(train_dataloader) - (batch_idx + 1)):0.4}'
        print(msg, flush=True)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Epoch: 01/02 | Batch: 00001/01887 | Loss: 0.6933 | Avg Loss: 0.6933 | Prediction: 0.031852830201387405 | Label: 1 | Time: 10.12 | Avg Time: 10.12 | Time Left: 1.909e+04
Epoch: 01/02 | Batch: 00002/01887 | Loss: 0.6947 | Avg Loss: 0.694 | Prediction: 0.024335898458957672 | Label: 0 | Time: 0.5774 | Avg Time: 5.35 | Time Left: 1.008e+04
Epoch: 01/02 | Batch: 00003/01887 | Loss: 0.6943 | Avg Loss: 0.6941 | Prediction: 0.018031645566225052 | Label: 0 | Time: 0.007185 | Avg Time: 3.569 | Time Left: 6.724e+03
Epoch: 01/02 | Batch: 00004/01887 | Loss: 0.6918 | Avg Loss: 0.6935 | Prediction: 0.011043533682823181 | Label: 1 | Time: 0.004329 | Avg Time: 2.678 | Time Left: 5.042e+03
Epoch: 01/02 | Batch: 00005/01887 | Loss: 0.6925 | Avg Loss: 0.6933 | Prediction: 0.011150605976581573 | Label: 0 | Time: 0.00422 | Avg Time: 2.143 | Time Left: 4.033e+03
Epoch: 01/02 | Batch: 00006/01887 | Loss: 0.694 | Avg Loss: 0.6934 | Prediction: 0.012746404856443405 | Label: 0 | Time: 0.004565 | Avg Time: 1.787 

Exception in thread Thread-14 (_pin_memory_loop):
Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/_utils/pin_memory.py", line 54, in _pin_memory_loop
    do_one_step()
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/_utils/pin_memory.py", line 31, in do_one_step
    r = in_queue.get(timeout=MP_STATUS_CHECK_INTERVAL)
  File "/usr/lib/python3.10/multiprocessing/queues.py", line 122, in get
    return _ForkingPickler.loads(res)
  File "/usr/local/lib/python3.10/dist-packages/torch/multiprocessing/reductions.py", line 495, in rebuild_storage_fd
    fd = df.detach()
  File "/usr/lib/python3.10/multiprocessing/resource_sharer.py", line 57, in detach
    with _resource_sharer.get_connection(self._id) as conn:
  File "/usr/lib/python3

In [ ]:
predictions = []
labels = []

model.eval()

for batch_idx, (data, targets) in enumerate(test_dataloader):

        outputs = model(data.to(device))
        predicted, _ = torch.max(outputs, 1)
        for i in range(len(predicted)):
          if predicted[i].item() < 0:
            predictions.append(0)
          else:
            predictions.append(1)
        labels.extend(targets.numpy())

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [ ]:
print(accuracy_score(labels, predictions))
print(confusion_matrix(labels, predictions))
print(classification_report(labels, predictions))